# Automatización de descarga de datos de Banxico

Para facilitar la descarga de las distintas series de tiempo que Banxico pone a disposición del público, se creó una Interfaz de programación de aplicaciones (API en inglés) en la cual un usuario puede acceder a la información de manera remota con sólo tener la clave de las series que desea consultar y un IDE (entorno de desarrollo integrado) con un lenguaje de programación capaz de interactuar con la API de Banxico, en este caso Python.

Para conocer los identificadores de serie de Banxico, favor de consultar la siguiente [liga](https://www.banxico.org.mx/SieAPIRest/service/v1/doc/catalogoSeries).

Para utilizar la API de series de tiempo de Banxico, es necesario primero **generar un token de consulta**, el cual se puede obtener en la siguiente [liga](https://www.banxico.org.mx/SieAPIRest/service/v1/token). 

Este token se genera una sola vez y se usa cuantas veces se necesite, respetando los límites de consulta (10,000 consultas al día y/o 200 consultas en 5 minutos por token). 

Se pueden descargar varias series a la vez en la misma función, siempre que sean de la misma periodicidad

In [ ]:
#series de prueba: TIIE 28: SF43783; inflación anual: SP30578; salario mínimo índice real: SL11297
#probar con varias series (mensuales, tienen que tener la misma periodicidad):SF283 (TIIE 28), SP30578 (inflación),SL11297 (salario mínimo)

### Función para descargar series de Banxico

In [3]:
import requests
import numpy as np
import pandas as pd

#pegar aquí el token que se generó en la liga anterior
token_1='82908162e488cf6a1e53620862cd2d9fa1181ba5a78882ffeb5d29320a19a145'

def descargar_banxico(serie,fecha_inicio,fecha_fin,token,es_oportuno=False):
    #Donde: 
    # serie: el identificador o identificadores que tiene Banxico para cada serie de tiempo en su sistema de información
    # fecha_inicio: fecha aaaa-mm-dd en la que se quiere iniciar la consulta
    # fecha_fin: fecha aaaa-mm-dd en la que se quiere concluir la consulta
    # token: el token generado por la API de Banxico para acceder

    if es_oportuno==True:
        url='https://www.banxico.org.mx/SieAPIRest/service/v1/series/'+serie+'/datos/oportuno?token='+token
    else:
        url='https://www.banxico.org.mx/SieAPIRest/service/v1/series/'+serie+'/datos/'+fecha_inicio+'/'+fecha_fin+'?token='+token
    headers = {'Token':token}
    response = requests.get(url,headers=headers)
    status=response.status_code
    raw_data = response.json()
    T=np.shape(serie.split(","))[0]
    info={'fecha':pd.DataFrame(raw_data['bmx']['series'][0]['datos'])['fecha']}

    for i in range(T):
        titulo=raw_data['bmx']['series'][i]['titulo']
        
        data=raw_data['bmx']['series'][i]['datos']
        
        df=pd.DataFrame(data)
        df['dato'] = df['dato'].apply(lambda x:float(x))
        info[titulo]=df['dato']
    info=pd.DataFrame(info)
    info.set_index('fecha',inplace=True)

    return info


,"Salarios Mínimos General Índice Real, Dic2018=100",Índice Nacional de Precios al consumidor variación anual
fecha,,
01/01/2016,95.2753,2.61
01/02/2016,94.8599,2.87
01/03/2016,94.7204,2.60
01/04/2016,95.0213,2.54
01/05/2016,95.4469,2.60
...,...,...
01/01/2023,189.9359,7.91
01/02/2023,188.8828,7.62
01/03/2023,188.3781,6.85


### Pruebas de la función de descarga

In [11]:
descargar_banxico('SP30578,SL11297','2016-01-01','2023-05-01',token_1)

,Índice Nacional de Precios al consumidor variación anual,"Salarios Mínimos General Índice Real, Dic2018=100"
fecha,,
01/01/2016,2.61,95.2753
01/02/2016,2.87,94.8599
01/03/2016,2.60,94.7204
01/04/2016,2.54,95.0213
01/05/2016,2.60,95.4469
...,...,...
01/01/2023,7.91,189.9359
01/02/2023,7.62,188.8828
01/03/2023,6.85,188.3781


In [12]:
descargar_banxico('SL11297','2016-01-01','2023-05-01',token_1)


,"Salarios Mínimos General Índice Real, Dic2018=100"
fecha,
01/01/2016,95.2753
01/02/2016,94.8599
01/03/2016,94.7204
01/04/2016,95.0213
01/05/2016,95.4469
...,...
01/01/2023,189.9359
01/02/2023,188.8828
01/03/2023,188.3781
